In [ ]:
!pip install requests
!pip install pandas
!pip install psycopg2
!pip install sqlalchemy


In [ ]:
import requests
import pandas as pd
import psycopg2
import json
from datetime import datetime
from sqlalchemy import create_engine

In [ ]:



url = "https://zoopla-real-time.p.rapidapi.com/properties/details"

querystring = {"listingId":"69200045"}

headers = {
	"x-rapidapi-key": "84f0978012msh1f18751f8e0d084p1e70c8jsn78ad1b993ee8",
	"x-rapidapi-host": "zoopla-real-time.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [ ]:
print(response.json())

In [ ]:
data = response.json()
file_name = 'zuma_data.json'
with open(file_name, 'w') as file:
    json.dump(data, file, indent = 4)



In [ ]:
zuma_df = pd.read_json('zuma_data.json')

In [ ]:
zuma_df.head()

In [ ]:
zuma_realestate_df = pd.read_json(r'C:\Users\Admin\Desktop\10Alytics\Realtor Project\zuma_realtor\Datasets\real_estate.json')

In [ ]:
zuma_realestate_df.head()

In [ ]:
zuma_realestate_df.isnull().sum()


In [ ]:
zuma_realestate_df['bedrooms'] = zuma_realestate_df['bedrooms'].fillna(0)
zuma_realestate_df['bathrooms']= zuma_realestate_df['bathrooms'].fillna(0)
zuma_realestate_df['propertyType'] = zuma_realestate_df['propertyType'].fillna('Not Specified')
zuma_realestate_df['legalDescription'] = zuma_realestate_df['propertyType'].fillna('Not Specified')

In [ ]:
zuma_realestate_df.info()

In [ ]:
zuma_realestate_df['bathrooms'].dtypes

In [ ]:
zuma_realestate_df['yearBuilt']= zuma_realestate_df['yearBuilt'].astype(str).replace('.0', '', regex=True)
zuma_realestate_df['features']= zuma_realestate_df['features'].astype(str)
zuma_realestate_df['id']= zuma_realestate_df['id'].astype(str)
zuma_realestate_df['owner']= zuma_realestate_df['owner'].astype(str)
zuma_realestate_df['propertyTaxes']= zuma_realestate_df['propertyTaxes'].astype(str)
zuma_realestate_df['bedrooms']= zuma_realestate_df['bedrooms'].astype('Int64')
zuma_realestate_df['bathrooms']= zuma_realestate_df['bathrooms'].astype('int64')
zuma_realestate_df['lastSaleDate ']= pd.to_datetime(zuma_realestate_df['lastSaleDate'])

In [ ]:
zuma_realestate_df.columns

In [ ]:
location_dim = zuma_realestate_df[['addressLine1','city','state','zipCode','formattedAddress','county','addressLine2']].copy().drop_duplicates().reset_index (drop=True)
location_dim['location_id']= range(1, len(location_dim)+1)
location_dim = location_dim[['location_id','addressLine1','city','state','zipCode','formattedAddress','county','addressLine2']]

In [ ]:
sales_dim = zuma_realestate_df[['lastSalePrice','lastSaleDate']].copy().drop_duplicates().reset_index (drop=True)
sales_dim['sales_id'] = range(1, len(sales_dim)+1)
sales_dim = sales_dim[['sales_id','lastSalePrice','lastSaleDate']]

In [ ]:
features_dim = zuma_realestate_df[['bedrooms', 'squareFootage', 'features', 'bathrooms', 'lotSize']].copy().drop_duplicates().reset_index (drop =True)
features_dim['feature_id'] = range(1, len(features_dim)+1)
features_dim = features_dim[['feature_id', 'bedrooms', 'squareFootage', 'features', 'bathrooms', 'lotSize']]

In [ ]:
zuma_fact = zuma_realestate_df.merge(location_dim, on = ['addressLine1','city','state','zipCode','formattedAddress','county','addressLine2'], how = 'left')\
                                .merge(sales_dim, on = ['lastSalePrice','lastSaleDate'], how = 'left')\
                                .merge(features_dim, on = ['bedrooms', 'squareFootage', 'features', 'bathrooms', 'lotSize'], how ='left')

In [ ]:
zuma_fact = zuma_fact [['id', 'location_id','sales_id', 'feature_id', 'yearBuilt', 'assessorID', 'legalDescription',
                                  'ownerOccupied','propertyType','taxAssessment','owner' ,'propertyTaxes', 'subdivision', 'zoning']]

In [ ]:
zuma_fact.head()

In [ ]:
location_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Realtor Project\zuma_realtor\Datasets\Transformed_Data\location_dim.csv', index = False)
sales_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Realtor Project\zuma_realtor\Datasets\Transformed_Data\sales_dim.csv', index = False)
features_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Realtor Project\zuma_realtor\Datasets\Transformed_Data\features_dim.csv', index = False)
zuma_fact.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Realtor Project\zuma_realtor\Datasets\Transformed_Data\zuma_fact.csv', index = False)

In [ ]:
zuma_fact.info()

In [ ]:
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        user ='postgres',
        password ='postgres',
        database ='zuma_realtor'
    )
    return connection

conn = get_db_connection()
cursor = conn.cursor()




In [ ]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    table_query =''' CREATE SCHEMA IF NOT EXISTS zuma;
    CREATE TABLE IF NOT EXISTS zuma.location_dim(
    location_id SERIAL PRIMARY KEY,
    addressLine1 VARCHAR(1000),
    city VARCHAR(1000),
    state VARCHAR(1000),
    zipCode INTEGER,
    formattedAddress VARCHAR(1000),
    county VARCHAR(1000),
    addressLine2 VARCHAR(1000)
    );

    CREATE TABLE IF NOT EXISTS zuma.sales_dim(
    sales_id SERIAL PRIMARY KEY,
    lastSalePrice FLOAT,
    lastSaleDate VARCHAR (1000)
    );

    CREATE TABLE IF NOT EXISTS zuma.features_dim(
    feature_id SERIAL PRIMARY KEY,
    bedrooms INTEGER,
    squareFootage FLOAT,
    features VARCHAR(1000),
    bathrooms INTEGER,
    lotSize FLOAT
    );

    CREATE TABLE IF NOT EXISTS zuma.zuma_fact(
    id VARCHAR(1000) PRIMARY KEY,
    location_id INTEGER,
    sales_id INTEGER,
    feature_id INTEGER,
    yearBuilt VARCHAR(1000),
    assessorID VARCHAR(1000),
    legalDescription VARCHAR(1000),
    ownerOccupied FLOAT,
    propertyType VARCHAR(1000),
    taxAssessment VARCHAR(1000),
    owner VARCHAR(1000),
    propertyTaxes VARCHAR(1000),
    subdivision VARCHAR(1000),
    zoning VARCHAR(1000),
    FOREIGN KEY (location_id) REFERENCES zuma.location_dim(location_id),
    FOREIGN KEY (sales_id) REFERENCES zuma.sales_dim(sales_id),
    FOREIGN KEY (feature_id) REFERENCES zuma.features_dim(feature_id)
    );

'''
    cursor.execute(table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
create_tables()

In [ ]:
zuma_fact.head()

In [ ]:
zuma_fact['taxAssessment'] = zuma_fact['taxAssessment'].astype(str)
zuma_fact['owner'] = zuma_fact['owner'].astype(str)
zuma_fact['propertyTaxes'] = zuma_fact['propertyTaxes'].astype(str)

In [ ]:

db_params ={
    'host': 'localhost',
    'user': 'postgres',
    'password':'postgres',
    'database': 'zuma_realtor',
    'port' : '5432'
}

db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

engine = create_engine(db_url)

with engine.connect() as connection:
    location_dim.to_sql('location_dim', connection, schema='zuma', if_exists='replace', index=False)
    sales_dim.to_sql('sales_dim', connection, schema='zuma', if_exists='replace', index=False)
    features_dim.to_sql('features_dim', connection, schema='zuma', if_exists='replace', index=False)
    zuma_fact.to_sql('zuma_fact', connection, schema='zuma', if_exists='replace', index=False)

